In [54]:
import torch
import numpy as np
from torchvision import datasets, transforms
from models.Nets import CNNMnist  # Import the correct model class
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from utils.options import args_parser  # Import the args_parser function
from sklearn.impute import SimpleImputer


In [44]:
# Temporarily override sys.argv
import sys
sys.argv = ['']  # Clears any command-line arguments

# Parse arguments
args = args_parser()

# Override specific arguments
args.model = 'cnn'
args.num_channels = 1
args.epochs = 5
args.num_users = 10
args.frac = 1.0

# Set device
args.device = torch.device('cuda' if args.gpu != -1 and torch.cuda.is_available() else 'cpu')


In [59]:
weights_path = "D:\Dawood_Work\Master_VT\Privacy_vs_Fairness\Privacy_vs_Fairness\Hypothesis_1\Chain-PPFL\weights\mnist_cnn_5_sample_dp_weights\model_weights_final.pth"

In [60]:
# Initialize the model
net = CNNMnist(args=args)  # Ensure args is defined and appropriate for your model
net.load_state_dict(torch.load(weights_path))
net.eval()  # Set the model to evaluation mode


CNNMnist(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [61]:
# Load the MNIST dataset
trans_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
dataset_train = datasets.MNIST('../data/mnist/', train=True, download=True, transform=trans_mnist)
dataset_test = datasets.MNIST('../data/mnist/', train=False, download=True, transform=trans_mnist)
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=args.local_bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=args.bs, shuffle=False)


In [62]:
# Step 1: Get the model's confidence scores for both training and test data
def get_confidence_scores(model, loader):
    model.eval()
    confidence_scores = []
    for data, target in loader:
        data, target = data.to(args.device), target.to(args.device)
        output = model(data)
        softmax_output = torch.softmax(output, dim=1)
        confidence, _ = torch.max(softmax_output, dim=1)
        confidence_scores.append(confidence.cpu().detach().numpy())
    return np.concatenate(confidence_scores)

In [78]:
# Get confidence scores for training and testing data
train_confidence = get_confidence_scores(net, train_loader)
test_confidence = get_confidence_scores(net, test_loader)

In [79]:
# Step 2: Create labels (1 for train, 0 for test)
train_labels = np.ones(len(train_confidence))
test_labels = np.zeros(len(test_confidence))

In [80]:
# Combine data
all_confidence = np.concatenate([train_confidence, test_confidence])
all_labels = np.concatenate([train_labels, test_labels])

In [81]:
# Step 3: Train an attack model (a simple logistic regression or MLP)
# Split the data into a train and validation set for the attack model
X_train, X_val, y_train, y_val = train_test_split(all_confidence.reshape(-1, 1), all_labels, test_size=0.5, random_state=args.seed)


In [82]:
# Train a simple logistic regression attack model
attack_model = LogisticRegression()
attack_model.fit(X_train, y_train)

LogisticRegression()

In [83]:
# Step 4: Evaluate the attack model
preds = attack_model.predict(X_val)
attack_success_rate = accuracy_score(y_val, preds)

In [84]:
print(f"Membership Inference Attack Success Rate: {attack_success_rate:.2f}")

Membership Inference Attack Success Rate: 0.86


In [ ]:
### DP

In [98]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from models.Nets import CNNMnist  # Ensure you have the correct model
import numpy as np

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the MNIST dataset
trans_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
dataset_train = datasets.MNIST('../data/mnist/', train=True, download=True, transform=trans_mnist)
train_loader = DataLoader(dataset_train, batch_size=64, shuffle=True)

# Initialize the model
model = CNNMnist(args)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Differential Privacy parameters
noise_multiplier = 5.0  # Controls the amount of noise added
clip_value = 1.0  # Clipping threshold for gradients

# Training loop with manual differential privacy
model.train()
for epoch in range(5):  # Adjust the number of epochs as needed
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()

        # Manually clip and add noise to gradients
        for param in model.parameters():
            if param.grad is not None:
                # Gradient clipping
                param.grad.data = torch.clamp(param.grad.data, -clip_value, clip_value)
                
                # Adding noise
                noise = torch.normal(0, noise_multiplier * clip_value, size=param.grad.data.size()).to(device)
                param.grad.data += noise

        optimizer.step()

    print(f"Epoch {epoch} completed.")

# Save the differentially private model
torch.save(model.state_dict(), 'dp_mnist_cnn_model_manual.pth')


Epoch 0 completed.
Epoch 1 completed.
Epoch 2 completed.
Epoch 3 completed.
Epoch 4 completed.


In [99]:
# Load the DP model
# model = CNNMnist().to(device)
# model.load_state_dict(torch.load('dp_mnist_cnn_model_manual.pth'))


model.eval()

# Perform MIA (same approach as discussed previously)


CNNMnist(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [100]:
net = model

In [101]:
# Load the MNIST dataset
trans_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
dataset_train = datasets.MNIST('../data/mnist/', train=True, download=True, transform=trans_mnist)
dataset_test = datasets.MNIST('../data/mnist/', train=False, download=True, transform=trans_mnist)
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=args.local_bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=args.bs, shuffle=False)

# Step 1: Get the model's confidence scores for both training and test data
def get_confidence_scores(model, loader):
    model.eval()
    confidence_scores = []
    for data, target in loader:
        data, target = data.to(args.device), target.to(args.device)
        output = model(data)
        softmax_output = torch.softmax(output, dim=1)
        confidence, _ = torch.max(softmax_output, dim=1)
        confidence_scores.append(confidence.cpu().detach().numpy())
    return np.concatenate(confidence_scores)

# Get confidence scores for training and testing data
train_confidence = get_confidence_scores(net, train_loader)
test_confidence = get_confidence_scores(net, test_loader)

# Step 2: Create labels (1 for train, 0 for test)
train_labels = np.ones(len(train_confidence))
test_labels = np.zeros(len(test_confidence))

# Combine data
all_confidence = np.concatenate([train_confidence, test_confidence])
all_labels = np.concatenate([train_labels, test_labels])

# Step 3: Train an attack model (a simple logistic regression or MLP)
# Split the data into a train and validation set for the attack model
X_train, X_val, y_train, y_val = train_test_split(all_confidence.reshape(-1, 1), all_labels, test_size=0.2, random_state=args.seed)

# Train a simple logistic regression attack model
attack_model = LogisticRegression()
attack_model.fit(X_train, y_train)

# Step 4: Evaluate the attack model
preds = attack_model.predict(X_val)
attack_success_rate = accuracy_score(y_val, preds)

print(f"Membership Inference Attack Success Rate: {attack_success_rate:.2f}")

Membership Inference Attack Success Rate: 0.86


In [106]:
import torch
import numpy as np
from torchvision import datasets, transforms
from models.Nets import CNNMnist  # Import the correct model class
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from utils.options import args_parser  # Import the args_parser function
import copy

# Parse arguments
args = args_parser()

# Override specific arguments
args.model = 'cnn'
args.num_channels = 1
args.epochs = 2
args.num_users = 10
args.frac = 1.0
args.device = torch.device('cuda' if args.gpu != -1 and torch.cuda.is_available() else 'cpu')

# Define the path to the saved DP model weights
weights_path = 'D:\Dawood_Work\Master_VT\Privacy_vs_Fairness\Privacy_vs_Fairness\Hypothesis_1\Chain-PPFL\weights\mnist_cnn_5_sample_dp_weights\model_weights_final.pth'

# Initialize the model
net = CNNMnist(args=args).to(args.device)
net.load_state_dict(torch.load(weights_path))
net.eval()  # Set the model to evaluation mode

# Load the MNIST dataset
trans_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
dataset_train = datasets.MNIST('../data/mnist/', train=True, download=True, transform=trans_mnist)
dataset_test = datasets.MNIST('../data/mnist/', train=False, download=True, transform=trans_mnist)
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=args.local_bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=args.bs, shuffle=False)

# Step 1: Get the model's confidence scores for both training and test data
def get_confidence_scores(model, loader, apply_dp=True):
    model.eval()
    confidence_scores = []
    w_noise = copy.deepcopy(model.state_dict())
    upsilon = 8  # Example DP noise factor, same as used in training
    epsilon = 1e-10  # Small value to avoid division by zero

    for data, target in loader:
        data, target = data.to(args.device), target.to(args.device)
        
        # Apply DP noise during evaluation
        if apply_dp:
            for lk in w_noise.keys():
                if 'weight' in lk:  # Apply noise only to weight layers
                    max_abs_val = torch.max(torch.abs(w_noise[lk]))
                    if max_abs_val > 0:
                        noised_sigma = float(2 * max_abs_val * args.local_ep) / float(args.epochs * upsilon + epsilon)
                    else:
                        noised_sigma = epsilon  # Prevent division by zero or very small scale

                    # Ensure noised_sigma is a valid number
                    if torch.isnan(torch.tensor(noised_sigma)) or noised_sigma <= 0:
                        noised_sigma = epsilon

                    distribution_laplace = torch.distributions.laplace.Laplace(0.0, noised_sigma)
                    w_noise[lk] += distribution_laplace.sample(w_noise[lk].size())
        
        # Copy DP-affected weights into the model
        model.load_state_dict(w_noise)
        output = model(data)
        softmax_output = torch.softmax(output, dim=1)
        confidence, _ = torch.max(softmax_output, dim=1)
        confidence_scores.append(confidence.cpu().detach().numpy())
    return np.concatenate(confidence_scores)

# Get confidence scores for training and testing data with DP applied

import numpy as np

# Function to check and handle NaN values
def handle_nan_values(X):
    if np.isnan(X).any():
        print("NaN values found in the input data. Handling NaN values...")
        # Option 1: Impute NaN values with the mean of the column
        X = np.nan_to_num(X, nan=np.nanmean(X))
        # Option 2: Remove rows with NaN values
        # X = X[~np.isnan(X).any(axis=1)]
    return X

# Get confidence scores for training and testing data with DP applied
train_confidence = get_confidence_scores(net, train_loader)
test_confidence = get_confidence_scores(net, test_loader)

# Combine data and handle NaN values
all_confidence = np.concatenate([train_confidence, test_confidence]).reshape(-1, 1)
all_confidence = handle_nan_values(all_confidence)

# Labels
train_labels = np.ones(len(train_confidence))
test_labels = np.zeros(len(test_confidence))
all_labels = np.concatenate([train_labels, test_labels])

# Split the data into a train and validation set for the attack model
X_train, X_val, y_train, y_val = train_test_split(all_confidence, all_labels, test_size=0.2, random_state=args.seed)

# Train a simple logistic regression attack model
attack_model = LogisticRegression()
attack_model.fit(X_train, y_train)

# Step 4: Evaluate the attack model
preds = attack_model.predict(X_val)
attack_success_rate = accuracy_score(y_val, preds)

print(f"Membership Inference Attack Success Rate: {attack_success_rate:.2f}")



NaN values found in the input data. Handling NaN values...
Membership Inference Attack Success Rate: 0.86
